In [ ]:
### Probabilistic Attempt with Pangu Weather (!! Actually not!! )
### For the ML based probabilistic models that WeatherBench 2 has (Pangu, Fuxi), it doesn't have ensemble forecasts for them?
### Only ensemble forecaster to test on is the ECMWF IFS ENS model (Physical numerically based)


In [1]:
import apache_beam

In [2]:
import os
import weatherbench2
import xarray as xr
import numpy as np
import math
from weatherbench2.regions import SliceRegion, ExtraTropicalRegion
from weatherbench2.evaluation import evaluate_in_memory
from weatherbench2 import config
from weatherbench2.metrics import CRPS, CRPSSpread, CRPSSkill, EnsembleVariance, EnsembleMeanMSE, EnsembleIgnoranceScore, EnsembleBrierScore, EnergyScore, EnergyScoreSkill, EnergyScoreSpread, RankHistogram 

In [3]:
forecast_path = 'gs://weatherbench2/datasets/ifs_ens/2018-2022-64x32_equiangular_conservative.zarr'
obs_path = 'gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_conservative.zarr'
climatology_path = 'gs://weatherbench2/datasets/era5-hourly-climatology/1990-2019_6h_64x32_equiangular_conservative.zarr'

In [4]:
climatology = xr.open_zarr(climatology_path)


In [22]:
forecasts = xr.open_zarr(forecast_path)
observations = xr.open_zarr(obs_path)


In [6]:
test = forecasts['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-12-31'))
ensembledim = test[0,:,0,0,0].size

In [29]:
paths1 = config.Paths(
    forecast=forecast_path,
    obs=obs_path,
    output_dir='./',   # Directory to save evaluation results
)

selection1 = config.Selection(
    variables=[
        'geopotential',
    ],
    levels=[500],
    time_slice=slice('2020-01-01', '2020-2-01'),
)

data_config1 = config.Data(selection=selection1, paths=paths1)

regions1 = {
    'global': SliceRegion(),
}

eval_configs1 = {
  'SpreadandSkill': config.Eval(
      metrics={
          #'CRPS': CRPS('number'),
          'CRPSSpread': CRPSSpread('number'),
          'CRPSSkill': CRPSSkill('number'),
          #'EnsembleVariance': EnsembleVariance('number'),
          #'EnsembleMeanMSE': EnsembleMeanMSE('number'),
          #'EnsembleIgnoranceScore': EnsembleIgnoranceScore('number'),
          #'EnsembleBrierScore': EnsembleBrierScore('number'),
          #'EnergyScore': EnergyScore('number'),
          #'EnergyScoreSkill': EnergyScoreSkill('number'),
          #'EnergyScoreSpread': EnergyScoreSpread('number'),
          #'RankHistogram': RankHistogram('number', 40),
      },
      regions=regions1
  )
}


evaluate_in_memory(data_config1, eval_configs1)



In [32]:
full = xr.open_dataset('./OnlyEnergyScore.nc')
mini = xr.open_dataset('./OnlyEnergyScoreMini.nc')
crpsmini = xr.open_dataset('./OnlyCRPSMini.nc')
spreadskill = xr.open_dataset('./SpreadandSkill.nc')

In [17]:
mini['geopotential'][:,:,:,:].values
full['geopotential'][:,:,:,:].values

array([[[[ 16.40059168],
         [ 17.37380748],
         [ 21.90202082],
         [ 22.90374028],
         [ 28.63360494],
         [ 31.81376322],
         [ 39.01819658],
         [ 44.04651572],
         [ 51.68230127],
         [ 57.91323458],
         [ 67.29016766],
         [ 74.74342197],
         [ 85.50607702],
         [ 94.58680874],
         [106.54174907],
         [116.78540843],
         [130.36353479],
         [141.70619667],
         [156.08422899],
         [168.37324771],
         [183.27989723],
         [195.86959851],
         [211.10127055],
         [224.03251049],
         [239.35082032],
         [252.27782218],
         [267.28129823],
         [279.85692916],
         [294.38048833],
         [306.46043771],
         [320.08435552],
         [331.38892106],
         [344.09150722],
         [354.43868065],
         [366.01286461],
         [375.52007359],
         [386.0933858 ],
         [394.6009482 ],
         [404.27010527],
         [411.86879601],


In [21]:
crpsmini['geopotential'][:,:,:,:].values

array([[[[ 15.39833045],
         [ 13.90813641],
         [ 18.48164672],
         [ 18.79502823],
         [ 23.33586116],
         [ 25.09675912],
         [ 29.79921545],
         [ 32.66286534],
         [ 37.49048854],
         [ 40.74957466],
         [ 46.30516335],
         [ 50.02635873],
         [ 56.65534858],
         [ 61.21588492],
         [ 69.04774343],
         [ 73.62024404],
         [ 82.33203342],
         [ 87.12597117],
         [ 96.89399812],
         [101.96509481],
         [112.14784674],
         [117.16877823],
         [127.53752385],
         [133.17227312],
         [143.99448775],
         [149.5352887 ],
         [160.33624455],
         [166.31286538],
         [176.7369237 ],
         [182.8505406 ],
         [192.72160336],
         [198.78255656],
         [208.19766894],
         [214.02217448],
         [222.52567799],
         [228.39327023],
         [236.27860973],
         [241.77518914],
         [248.94896831],
         [254.17263142],


In [60]:
skill = spreadskill['geopotential'].sel(metric = 'CRPSSkill')
spread = spreadskill['geopotential'].sel(metric = 'CRPSSpread')

#print(skill.values-spread/2)
print(skill.values)
spread.values


[[[ 23.73866304]
  [ 23.90021249]
  [ 30.91322866]
  [ 34.05420002]
  [ 42.00982705]
  [ 47.46495631]
  [ 56.17729284]
  [ 63.13925993]
  [ 72.27117911]
  [ 80.0075033 ]
  [ 90.22649832]
  [ 98.77403801]
  [110.4483304 ]
  [120.3977575 ]
  [133.85186203]
  [144.30469093]
  [159.08652682]
  [170.19169643]
  [186.51491316]
  [198.39449839]
  [215.58971092]
  [227.77626476]
  [245.4786898 ]
  [258.54356641]
  [276.92603696]
  [290.1243808 ]
  [308.71052312]
  [322.39835119]
  [340.54700531]
  [354.34626372]
  [371.89054464]
  [385.48179663]
  [402.24125238]
  [415.18946598]
  [430.66399729]
  [443.29428707]
  [457.68082197]
  [469.427438  ]
  [482.65964666]
  [493.65758194]
  [505.76101657]
  [515.75075475]
  [526.55196781]
  [535.46755852]
  [545.40918455]
  [553.58284309]
  [562.9220238 ]
  [570.3904321 ]
  [579.03124833]
  [585.38735445]
  [592.98249733]
  [598.39624532]
  [605.58963884]
  [610.42218369]
  [617.00861506]
  [621.13669276]
  [627.43843726]
  [631.36723003]
  [637.2948839

array([[[ 16.68066519],
        [ 19.98415216],
        [ 24.86316387],
        [ 30.51834358],
        [ 37.34793178],
        [ 44.73639437],
        [ 52.75615478],
        [ 60.95278919],
        [ 69.56138113],
        [ 78.51585729],
        [ 87.84266995],
        [ 97.49535857],
        [107.58596364],
        [118.36374517],
        [129.60823719],
        [141.36889377],
        [153.5089868 ],
        [166.13145052],
        [179.24183006],
        [192.85880715],
        [206.88372837],
        [221.21497306],
        [235.8823319 ],
        [250.74258659],
        [265.86309841],
        [281.1781842 ],
        [296.74855714],
        [312.17097162],
        [327.62016324],
        [342.99144624],
        [358.33788257],
        [373.39848015],
        [388.0871669 ],
        [402.334583  ],
        [416.27663858],
        [429.80203369],
        [442.80442446],
        [455.30449773],
        [467.4213567 ],
        [478.96990104],
        [490.09370205],
        [500.760

In [28]:
latitude = forecasts['latitude'][:].values
delta = 2.8125
theta_upper = latitude + delta
theta_lower = latitude - delta

# Calculate weights based on the provided formula
weights = (np.sin(np.radians(theta_upper)) - np.sin(np.radians(theta_lower)))
weights /= weights.sum()
weights *= 32

weights

array([0.07704437, 0.23039114, 0.38151911, 0.52897285, 0.67133229,
       0.80722643, 0.93534654, 1.05445875, 1.16341595, 1.26116882,
       1.34677594, 1.41941287, 1.47838008, 1.52310968, 1.55317091,
       1.56827425, 1.56827425, 1.55317091, 1.52310968, 1.47838008,
       1.41941287, 1.34677594, 1.26116882, 1.16341595, 1.05445875,
       0.93534654, 0.80722643, 0.67133229, 0.52897285, 0.38151911,
       0.23039114, 0.07704437])

In [ ]:
#Testing on Mini, code not generalised

#15 extra days lead
a = forecasts['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-02-01'))[:,:,:,:]
b = observations['geopotential'].sel(level=500, time = slice('2020-01-01', '2020-02-16'))[:,:,:]

#Skill
skills = []
for i in range(61):
    ashifted = a.assign_coords(time = a.time + a[:,:,i,:,:]['prediction_timedelta'].values)
    if (i ==60):
        vals = np.sum(abs(ashifted[:,:,i,:,:]-b[i:,:,:][::2,:,:]) *weights[None,None,:], axis = (0,1,2,3))/(64*50*64*32)
    else:
        vals = np.sum(abs(ashifted[:,:,i,:,:]-b[i:i-60,:,:][::2,:,:]) *weights[None,None,:], axis = (0,1,2,3))/(64*50*64*32)
    skills.append(vals.values)

skills


[array(23.73866304),
 array(23.90021249),
 array(30.91322866),
 array(34.05420002),
 array(42.00982705),
 array(47.46495631),
 array(56.17729284),
 array(63.13925993),
 array(72.27117911),
 array(80.00750331),
 array(90.22649833),
 array(98.77403801),
 array(110.44833038),
 array(120.39775749),
 array(133.85186204),
 array(144.30469092),
 array(159.08652681),
 array(170.19169643),
 array(186.51491313),
 array(198.39449838),
 array(215.58971091),
 array(227.77626475),
 array(245.47868984),
 array(258.54356644),
 array(276.92603697),
 array(290.12438084),
 array(308.71052315),
 array(322.39835109),
 array(340.54700529),
 array(354.34626371),
 array(371.89054462),
 array(385.48179658),
 array(402.24125224),
 array(415.18946574),
 array(430.66399708),
 array(443.29428712),
 array(457.68082199),
 array(469.42743805),
 array(482.65964672),
 array(493.65758205),
 array(505.7610166),
 array(515.75075474),
 array(526.55196797),
 array(535.46755875),
 array(545.40918452),
 array(553.58284313),
 

In [ ]:
#Page 12 of WeatherBench 2 paper suggests more efficient way to calculate E[X-X'], using loops, not done here. 
#Spread 

Spreads = []

for i in range(61):
    ashifted = a.assign_coords(time = a.time + a[:,:,i,:,:]['prediction_timedelta'].values)
    if (i ==60):
        vals = np.sum(abs(ashifted[:,:,i,:,:]-b[i:,:,:][::2,:,:]) *weights[None,None,:], axis = (0,1,2,3))/(64*50*64*32)
    else:
        vals = np.sum(abs(ashifted[:,:,i,:,:]-b[i:i-60,:,:][::2,:,:]) *weights[None,None,:], axis = (0,1,2,3))/(64*50*64*32)
    Spreads.append(vals.values)

skills

In [ ]:
#Page 12 of WeatherBench 2 paper suggests more efficient way to calculate E[X-X'], using loops, not done here. 
#However, this loop method for just lag 0 takes 6 minutes.
#
# Spread 
#

a = forecasts['geopotential'].sel(level = 500, time = slice('2020-01-01', '2020-02-01'))[:,:,:,:]
weighted_abs_differences = xr.zeros_like((abs(a[:, 0,0, :, :] - a[:, 1,0, :, :]) * weights[None,None,:]))

for m in range(50):
    for n in range(50):
        if m != n:
            weighted_abs = abs(a[:, m,0, :, :] - a[:, n,0, :, :]) * weights[None,None,:]
            weighted_abs_differences += weighted_abs


final_metric = np.sum(weighted_abs_differences) / (50*(50 - 1)*64*64*32)  


In [ ]:
#Takes ~6 Minutes to run, but does calculate spread correctly... 
final_metric.values

array(16.68066519)

In [ ]:
#Have had some work with probabilistic diagnostics, but time demands are increasing. Going to try experimenting with sigkernel now

In [254]:
import torch
import sigkernel

In [ ]:
# Specify the static kernel (for linear kernel use sigkernel.LinearKernel())
static_kernel = sigkernel.RBFKernel(sigma=2)

# Specify dyadic order for PDE solver (int > 0, default 0, the higher the more accurate but slower)
dyadic_order = 1

# Specify maximum batch size of computation; if memory is a concern try reducing max_batch, default=100
max_batch = 100

# Initialize the corresponding signature kernel
signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)

# Synthetic data
batch, len_x, len_y, dim = 5, 10, 20, 2
X = torch.rand((batch,len_x,dim), dtype=torch.float64, device='cuda') # shape (batch,len_x,dim)
Y = torch.rand((batch,len_y,dim), dtype=torch.float64, device='cuda') # shape (batch,len_y,dim)
Z = torch.rand((batch,len_x,dim), dtype=torch.float64, device='cuda') # shape (batch,len_y,dim)



# Compute signature kernel "batch-wise" (i.e. k(x_1,y_1),...,k(x_batch, y_batch))
K = signature_kernel.compute_kernel(X,Y,max_batch)

# Compute signature kernel Gram matrix (i.e. k(x_i,y_j) for i,j=1,...,batch), also works for different batch_x != batch_y)
G = signature_kernel.compute_Gram(X,Y,max_batch)

G
# Compute MMD distance between samples x ~ X and samples y ~ Y, where X,Y are two distributions on path space...
mmd = signature_kernel.compute_mmd(X,Y,max_batch)

mmd
# # ... and to backpropagate through the MMD distance simply call .backward(), like any other PyTorch loss function
#mmd.backward()

# # Compute scoring rule between X and a sample path y, i.e. S_sig(X,y) = E[k(X,X)] - 2E[k(X,y] ...
# y = Y[0]
# sr = signature_kernel.compute_scoring_rule(X,y,max_batch)

# # ... and expected scoring rule between X and Y, i.e. S(X,Y) = E_Y[S_sig(X,y)]
# esr = signature_kernel.compute_expected_scoring_rule(X,Y,max_batch)

# # Sig CHSIC: XY|Z
# sigchsic = signature_kernel.SigCHSIC(X, Y, Z, static_kernel, dyadic_order=1, eps=0.1)

True


tensor(0.0042, device='cuda:0', dtype=torch.float64)

In [291]:
import torch
import sigkernel

# Specify the static kernel (for linear kernel use sigkernel.LinearKernel())
static_kernel = sigkernel.RBFKernel(sigma=1.5)

# Specify dyadic order for PDE solver (int > 0, default 0, the higher the more accurate but slower)
dyadic_order = 2

# Specify maximum batch size of computation; if memory is a concern try reducing max_batch, default=100
max_batch = 100

# Initialize the corresponding signature kernel
signature_kernel = sigkernel.SigKernel(static_kernel, dyadic_order)

# Synthetic data
batch, len_x, len_y, dim = 5, 10, 20, 2
X = torch.rand((batch,len_x,dim), dtype=torch.float64, device='cuda') # shape (batch,len_x,dim)
Y = torch.rand((batch,len_y,dim), dtype=torch.float64, device='cuda') # shape (batch,len_y,dim)
Z = torch.rand((batch,len_x,dim), dtype=torch.float64, device='cuda') # shape (batch,len_y,dim)

# Compute signature kernel "batch-wise" (i.e. k(x_1,y_1),...,k(x_batch, y_batch))
K = signature_kernel.compute_kernel(X,Y,max_batch)

# Compute signature kernel Gram matrix (i.e. k(x_i,y_j) for i,j=1,...,batch), also works for different batch_x != batch_y)
G = signature_kernel.compute_Gram(X,Y,sym=False)

# Compute MMD distance between samples x ~ X and samples y ~ Y, where X,Y are two distributions on path space...
mmd = signature_kernel.compute_mmd(X,Y,max_batch)
# ... and to backpropagate through the MMD distance simply call .backward(), like any other PyTorch loss function
mmd.backward()

mmd
# # Compute scoring rule between X and a sample path y, i.e. S_sig(X,y) = E[k(X,X)] - 2E[k(X,y] ...
y = Y[0]

sr = signature_kernel.compute_scoring_rule(X,y,max_batch)

# # ... and expected scoring rule between X and Y, i.e. S(X,Y) = E_Y[S_sig(X,y)]
# esr = signature_kernel.compute_expected_scoring_rule(X,Y,max_batch)

# # Sig CHSIC: XY|Z
# sigchsic = signature_kernel.SigCHSIC(X, Y, Z, static_kernel, dyadic_order=1, eps=0.1)

c:\Users\arche\anaconda3\envs\Diss\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 5 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
c:\Users\arche\anaconda3\envs\Diss\Lib\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 25 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn